In [1]:
import numpy as np
import matplotlib.pyplot as pyplot
import pandas as pd
import scipy.sparse as sps
%matplotlib inline
%load_ext Cython

from Data_manager.split_functions.split_train_validation_random_holdout import split_train_in_two_percentage_global_sample
from Evaluation.Evaluator import EvaluatorHoldout
from Recommenders.SLIM.SLIM_BPR_Python import SLIM_BPR_Python
import optuna

C:\Users\feder\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# data_train_path="/kaggle/input/recommender-system-2023-challenge-polimi/data_train.csv"
# data_target_user_path="/kaggle/input/recommender-system-2023-challenge-polimi/data_target_users_test.csv"
data_train_path="data_train.csv"
data_target_user_path="data_target_users_test.csv"
data_train = pd.read_csv(data_train_path)
data_target = pd.read_csv(data_target_user_path)

In [3]:
URM_all =  data_train.pivot(index='row', columns='col', values='data').fillna(0)
item_map = {i : item for i, item in enumerate(URM_all.columns)}
user_map = {i : user for i, user in enumerate(data_target["user_id"])}
item_map_inv = {item : i for i, item in item_map.items()}
user_map_inv = {user : i for i, user in user_map.items()}
missing_index = [x for x in range(1,13025) if x not in URM_all.index.tolist()]
add_urm = pd.DataFrame(index = missing_index, columns = URM_all.columns).fillna(0)
URM_all = pd.concat([URM_all, add_urm]).sort_index()
del add_urm
del missing_index
#data_target["user_id"] = data_target["user_id"]
URM_all = URM_all.to_numpy()
URM_all = sps.csr_matrix(URM_all)
URM_all

<13024x22222 sparse matrix of type '<class 'numpy.float64'>'
	with 478730 stored elements in Compressed Sparse Row format>

In [4]:
test_folds = 3

URM_trains = [None] * test_folds
evaluator_tests = [None] * test_folds
for i in range(0, test_folds):
    URM_trains[i], URM_test = split_train_in_two_percentage_global_sample(URM_all, train_percentage = 0.80)
    evaluator_tests[i] = EvaluatorHoldout(URM_test, cutoff_list=[10])

EvaluatorHoldout: Ignoring 2560 (19.7%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 2552 (19.6%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 2553 (19.6%) Users that have less than 1 test interactions


In [5]:
def train_evaluate(optuna_trial):
    topk = optuna_trial.suggest_int("topK", 1, 1000)
    lambda_i = optuna_trial.suggest_float("lambda_i", 1e-5, 1e-1, log=True)
    lambda_j = optuna_trial.suggest_float("lambda_j", 1e-6, 1e-2, log=True)
    learning_rate = optuna_trial.suggest_float("learning_rate", 1e-5, 1e-1, log=True)

    mAP = 0.0
    for i in range(0, test_folds):
        SLIM_recommender = SLIM_BPR_Python(URM_trains[i], verbose=False)
        SLIM_recommender.fit(topK=topk, lambda_i=lambda_i, lambda_j=lambda_j, learning_rate=learning_rate)
        result_df, _ = evaluator_tests[i].evaluateRecommender(SLIM_recommender)
        mAP += result_df["MAP"].values[0]
    mAP /= test_folds
    return mAP

In [6]:
study = optuna.create_study(direction="maximize")
study.optimize(train_evaluate, n_trials=500)

[I 2023-12-11 22:31:57,666] A new study created in memory with name: no-name-288bda15-6a1a-437d-8350-3e2cca2b30ad


Recommender_Base_Class: URM Detected 592 ( 4.5%) users with no interactions.
Recommender_Base_Class: URM Detected 99 ( 0.4%) items with no interactions.


[W 2023-12-11 22:32:40,267] Trial 0 failed with parameters: {'topK': 317, 'lambda_i': 0.0001525226452236062, 'lambda_j': 2.278488632215289e-05, 'learning_rate': 0.0009201421148193838} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "C:\Users\feder\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\optuna\study\_optimize.py", line 200, in _run_trial
    value_or_values = func(trial)
                      ^^^^^^^^^^^
  File "C:\Users\feder\AppData\Local\Temp\ipykernel_17332\1877965180.py", line 10, in train_evaluate
    SLIM_recommender.fit(topK=topk, lambda_i=lambda_i, lambda_j=lambda_j, learning_rate=learning_rate)
  File "c:\Users\feder\Documents\DEV\my_RecSyS_Course_AT_PoliMi\Recommenders\SLIM\SLIM_BPR_Python.py", line 53, in fit
    self._run_epoch(n_epoch)
  File "c:\Users\feder\Documents\DEV\my_RecSyS_Course_AT_PoliMi\Recommenders\SLIM\SLIM_BPR_Python.py", line 7

KeyboardInterrupt: 